<h1>Temporal Difference Methods For Prediction of State Value Function: Using On Policy One-Step TD(0) for Windy Gridworld</h1>


Temporal Difference methods are another class of tabular solutions which can be used to estimate the state value function $V(S_{t})$ described by the Bellman eauations.[[4]](#References) This notebook contains functions that estimate $V(S_{t})\approx v_{\pi}(s_{t})$ using the One Step TD(0) algorithm for the Windy Gridworld problem described in [[5]](#References).

This notebook will require the following Python modules:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.image as mpimg
from IPython.display import Image
import seaborn as sns
import import_ipynb
import ipynb.fs.full.envWindyGridworld as wgw
import ipynb.fs.full.targetPolicyWindyGridworld as tPolicy
import ipynb.fs.full.returnsStateWindyGridworld as returnsStateWindyGridworld
import ipynb.fs.full.stateValueWindyGridworld as stateValueWindyGridworld


import ipynb.fs.full.helpers as h

ModuleNotFoundError: No module named 'matplotlib'

<h2>Environment Windy Gridworld</h2>

Put a picture of the Windy Gridworld maze here...

The Windy Gridworld environment is composed of a class containing the maze itself, the rules for wind effects within the maze and helper functions to compute the rewards associated with any of the available actions. See the [Windy Gridworld environment class notebook](envWindyGridworld.ipynb) for details.

<h3>Environment Windy Gridworld: Maze</h3>

The maze is represented as a 20x20 numpy array of integers where each elelment of the 2D array represents a position in the maze and the value of each element signifies whether the position can be occupied by the agent (i.e. position is not a wall), and whether the position is "windy" in one in any of the possible directions. Landing on a "windy" position in the maze results in the agent being blown into the nearest wall that lies down wind from the agent.

$$\large maze[i,j]
    = 
    \begin{cases}
    -1\quad\,\,\,\,\,\,\,\,\normalsize\text{position is a wall} \\
    0\quad\quad \,\,\,\,\,\,\normalsize\text{position is not a wall and wind is not blowing } \\
    1\quad\quad \,\,\,\,\,\,\normalsize\text{position is not a wall and wind blows N } \\
    2\quad\quad \,\,\,\,\,\,\normalsize\text{position is not a wall and wind blows E} \\
    3\quad\quad \,\,\,\,\,\,\normalsize\text{position is not a wall and wind blows S} \\
    4\quad\quad \,\,\,\,\,\,\normalsize\text{position is not a wall and wind blows W} 
    \end{cases}
$$

A predifined maze has been saved as a 2D numpy array. Let's load the 2D numpy array.

In [15]:
maze = np.load('./maze.npy')

In [16]:
maze

array([[ 0, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
         0, -1,  0,  0],
       [ 0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
         0, -1,  0,  0],
       [ 0,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         0, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, -1,  0,  0],
       [ 0,  0, -1,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  3, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,
         0,  0,  0, -1],
       [ 0,  0, -1,  0,  0,  3, -1,  0,  0, -1, -1, -1,  0,  0,  0, -1,
        -1, -1, -1,  0],
       [ 0,  0, -1,  0,  0,  3, -1,  0,  0,  0,  0, -1,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  3,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 0,  0,  0, -1, -1, -1

Notice that the positions (4, 5), (5, 5), (6, 5), (7, 5), (8, 5) and (9, 5) are all "windy" and blow in the southern direction since each of their states is 3.

<h3>Environment Windy Gridworld: State Space</h3>

Each white square in the maze represnts a state which is a random variable denoted as $S$.
Each observed state $s\in S$ is defined as an ordered pair $(x, y)$ which represents the position of the agent on the grid.

$$\large S\coloneqq\{(x, y): (x\in\mathbb{Z}) \cap (0\le x<n) \bigcap (y\in\mathbb{Z}) \cap (0\le y<m)\}$$

(20, 20)

<h3>Environment Windy Gridworld: Action Space

Each action is a random variable which we denote with the symbol $A$ and there are four actions available to the agent.


$$\large A\coloneqq\{Up, Down, Left, Right\}$$

<h3> Environment Windy Gridworld: Episodes and Time Steps

In Gridworld, an episode consists of a discrete sequence of time steps that occur from the time when the agent first begins, at the "Start" square on our grid, to the time at which our agent finally reaches the "Finish" square on the grid.
    
The time step is denoted by the variable $t$ and is set to zero at the beginning of the first episode. At this time our agent is located in the "Start" square on the grid.

The time step variable $t$ is then incremented by one after each action is take by our agent until it reaches the "Finish" sqaure on the grid.

The variable $T$ is used to represent the value of the time step $t$ upon which our agent reaches the "Finish" square on the grid and the episode ends.

If another episode is carried out, the agent returns to the "Start" square and the time step variable $t$ is set to $T+1$.

<h3>Environment Windy Gridworld: Rewards and Returns

<h3>Environment Gridworld: Rewards</h3>

At each time step the reward of an action taken is either -1 or 0 depending upon whether the next state is a finish line or non-finish line square on the grid. 

$$
    \large R_t
    =
    \begin{cases}
    0\quad\quad \Large s_{t+1}\normalsize =\text{ finish line square on the grid } \\
    -1\quad\, \Large s_{t+1}\normalsize\neq\text{ finish line square on the grid}
    \end{cases}
    $$

<h3>State Value Function</h3>

According to the Bellman eqautions, the state value function at time $t$, $v_{\pi}(s_{t})$, within an episode represents the expected return when in state $s_{t}\in S$ according to policy $\pi$. [\[1\]](#References)

$\large\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad v_\pi(s_t)\coloneqq\mathbb{E} [G_t | S_t=s_t]$<br>
$\large\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad=\mathbb{E} [R_{t+1} + \gamma G_{t+1} | S_t=s_t]$<br>
$\large\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad=\mathbb{E} [R_{t+1} + \gamma v_{\pi}(S_{t+1})\,\,|\,\,S_{t}=s_{t}]$<br>


Unlike Monte Carlo state value prediction methods, TD methods update the estimate of the state value function at each time-step within an episode (instead of updating the estimate at the end of each episode).

$$\large V(S_{t})\leftarrow V(S_{t})+\alpha[R_{t+1}+\gamma V(S_{t+1})-V(S_{t})])]$$

An $mxn$ matrix $V$ is used to store the values of each state $v_\pi(s)$ on the grid where, according to the Bellman equations, the state value represents the expected return of the current state while folowing policy $\pi.$[[4]](#References)

<h2>Policy</h2>

Our agent's policy is denoted as $\pi$ and is used to represent the conditional probability mass function $f_{A|S}$ of actions over the states:<br>
    $$\large \pi\coloneqq f_{A|S}$$
    At any given timestep $t$, our agent must choose an action $a\in A$ to move from its current state $s\in S$ to its next state $s'\in S$.
    We denote the conditional probability of our agent selecting action $a\in A$ given its current state $s\in S$ by the following notation:
    $$\large \pi(a|s)=f_{A|S}(a|s)=P(A=a|S=s)$$

A policy used in Windy Gridworld is represented by an $mxnx|S|$ numpy array pi where each element represents the probability of taking action $a\in A$ while in the state $s=(i,j)$
   ]

As an example a random policy, example_pi, for taking actions $A$ with equal probability in a $2x2$ maze would have the shape (2,2,4).

In [ ]:
example_pi = np.zeros((2,2,4),dtype=np.float32)
example_pi[:] = 0.25
print(example_pi)
example_pi.shape

[[[0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]]

 [[0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]]]


(2, 2, 4)

In [ ]:
del example_pi

<h2>State Value Function</h2>

As an example the 2D numpy array V for a maze of shape $2x2$ represents the state value $v_\pi(s)$ for each $s\in S$ and would have the shape (2,2).

In [ ]:
example_V = np.random.sample((2,2))
print(example_V)
example_V.shape

[[0.1345076  0.83796436]
 [0.98025865 0.94497563]]


(2, 2)

In [ ]:
del example_V

<h3>TD(0) Prediction State Value Algorithm</h3> 

Psuedocode for the TD(0) Prediction State Value algorithm which estimates the state value function $V(s_t)$ is found below and as described in Barto et. al. [[1]](#References)

$\large\quad\quad\text{TD(0) Prediction State Value Algorithm}$<br>

$\large\quad\quad\text{Inputs:}\,\,\text{target policy\,\,}\pi,\,\,\,\text{environment},\,\,\text{stepSize}\,\alpha,\,\,\text{discount factor }\gamma,\,\,\text{tolerance },\,\,\,\text{numEpisodes }$<br>
$\large\quad\quad\text{ 1.}\quad V(terminal)\leftarrow0,\,V(s)\leftarrow\text{Randomly chosen state value for all non-terminal states }$<br>
$\large\quad\quad\text{ 2.}\quad\text{Loop forever (for each episode)}:$<br>
$\large\quad\quad\text{ 3.}\quad\quad\text{Initialize }S(0)=s$<br>
$\large\quad\quad\text{ 4.}\quad\quad\text{Loop for each step of episode}:$<br>
$\large\quad\quad\text{ 5.}\quad\quad\quad A\leftarrow\text{ action given by policy }\pi\text{ for }S(0)$<br>
$\large\quad\quad\text{ 6.}\quad\quad\quad\text{Take action }A\text{, observe }R\text{ and }S_{t+1}$<br>
$\large\quad\quad\text{ 7.}\quad\quad\quad V(S_{t})\leftarrow\,V(S_{t})\,+\alpha[R+\gamma V(S_{t+1})\,-\,V(S_{t})]$<br>
$\large\quad\quad\text{ 8.}\quad\quad\quad S\leftarrow S_{t+1}$<br>
$\large\quad\quad\text{ 9.}\quad\quad\quad\text{until }S\text{ is terminal}$<br>
$\large\quad\quad\text{10.}\quad\quad\text{Return }V$<br>

This simple one-step TD method is called TD(0), which is a special case of the bootstrapping method called TD($\gamma$), and combines the sampling strategies of Monte Carlo methods with the bootstrapping used in Dynamic Programming methods. [[1]](#References)[[2]](#References) 

In [ ]:
def TemporalDifferencePrediciton_StateValue_TD_0_WindyGridWorld(startPosition, targetPolicy, environment, stepSize, discountFactor, tolerance, numEpisodes):
    # 1. Initialize V(terminal) to zero and all other V(s) to random state value
    V = np.random.sample((environment.Maze.shape[0], environment.Maze.shape[1]))
    # 1. Set the state value of walls within the maze to -1, and set the finish position state value to 0
    for i in range(environment.Maze.shape[0]):
        for j in range(environment.Maze.shape[1]):
            if (environment.Maze[i,j] == -1):
                V[i,j] = (float)('-inf')
                continue
            if (i == environment.finish_position[0] and j == environment.finish_position[1]):
                V[i,j] = 0
    # 2. Loop forever or for each episode
    for _ in range(numEpisodes):
        # 3. Intitialize the temporary placeholder S with 
        S = startPosition        
        # 4. Loop over each step in the episode
        isDone = False
        while( not(isDone) ):
              # 5. A is set to the action chosen for the current state given the policy
              A = {}
              # 6. Take action a and observe R and S_{t+1}

              # 7. Update the estimate of V(S_{t})
              V = {}
              # 8. Set S to S_{t+1}
              S = {}

    return V

<h3>Perform TD(0) for prediction of state value function</h3>

In [ ]:

# Set the step size alpha for updating the estimate of the state value function
alpha = 0.1
# Set the discount factor gamma for rewards
gamma = 0.99

# Set a tolerance for comparison of updated state values for state value estimation
tolerance = 0.0000000001

# Select a square in the Windy Gridworld maze to act as the starting line square
start_position = (19, 0)

# Select a square in the Windy Gridworld maze to act as the finish line square
finish_position = (0,19)

In [ ]:
# Set teh number of runs
numRuns = 1
# episodes [ [1, 10, 50, 100, 200, 500, 1000, 3000, 5000, 10000]
episodes = 100

An environment can be instantiated by simply choosing a finish position and then calling the envGridworld constructor which takes a path to a numpy array representing the maze and the selected finishing square position as a tuple.

Now run the TD(0) algorithm on the Windy Gridworld environmnet object using the parameters set above.

In [ ]:
# Run the experiment
for i in range(len(episodes)):
    # Run an experiment numRuns times
    for runNum in range(numRuns):
        # Set the environment
        env = wgw.envWindyGridworld(path_to_maze="./maze.npy", start_position = start_position, finish_position = finish_position)
        # Open a pre-trained target policy that was serialized to json
        with open('policies/dumbPolicy.pickle', 'rb') as handle:
            pi = pickle.load(handle)
        # Instatiate and initialize a new state value function V(S_t)
            V_s = stateValue.stateValueWindyGridworld(staetSpace=stateSpaceWindyGridworld, actionSpace=actionSpaceWindyGridworld)
        # Instantiate and uinitialize a new returns runction R(S_t) with empty lists
            R_s = returnsS
        # Run the TD(0) prediciton algorithm
        v_star_estimate, path_to_state_value_fig =  TemporalDifferencePrediciton_StateValue_TD_0_WindyGridWorld(targetPolicy=pi, environment=env, V_s=V_s, R_s=R_s, stepSize=alpha, discountFactor=gamma, numEpisodes=numEpisodes)

<h3>TD(0) Estimation of State Value Function Results for Windy Gridworld</h3>

The resulting estimates for the state value function for all states in the Windy Gridworld environment are provided below where each position in the maze contains:
1. the direction of the wind (if any)
2. the estimated state value $v_{\pi}$ of each position on the grid while following policy $\pi$

In [ ]:
Image(filename=path_to_state_value_fig)

<h2>References</h2>

1. Sutton R.S., Barto A.G., "Reinforcement Learning, An Introduction", 2nd ed, MIT Press, Cambridge MA, 2018, p. 120.
2. Sutton R.S., Barto A.G., "Reinforcement Learning, An Introduction", 2nd ed, MIT Press, Cambridge MA, 2018, pp. 142-145.
3. Sutton R.S., Barto A.G., "Reinforcement Learning, An Introduction", 2nd ed, MIT Press, Cambridge MA, 2018, pp. 287-301.
4. Sutton R.S., Barto A.G., "Reinforcement Learning, An Introduction", 2nd ed, MIT Press, Cambridge MA, 2018, p. 58.
5. Sutton R.S., Barto A.G., "Reinforcement Learning, An Introduction", 2nd ed, MIT Press, Cambridge MA, 2018, p. 130.

